In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_train.npy")
label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_train.npy")

In [ ]:
def tf_dataset(x,y,batch_size):
    dataset=tf.data.Dataset.from_tensor_slices((x,y))
    dataset=dataset.shuffle(buffer_size=64)
    dataset=dataset.batch(batch_size)
    dataset=dataset.prefetch(tf.data.AUTOTUNE)
    return dataset
dataset_train=tf_dataset(data, label, batch_size=64)

In [ ]:
dataset_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180), dtype=tf.float32, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>

In [ ]:
num_classes = 5

model = Sequential([
  layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
  layers.MaxPool1D(),
  layers.Conv1D(32, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Conv1D(64, 3, padding='same', activation='relu'),
  layers.MaxPool1D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes,activation="softmax")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 180, 16)           64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 90, 16)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 90, 32)            1568      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 45, 32)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 45, 64)            6208      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 22, 64)           0

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(dataset_train, epochs=50)

Epoch 1/50
1198/1198 [==============================] - 18s 5ms/step - loss: 0.2524 - categorical_accuracy: 0.9298
Epoch 2/50
1198/1198 [==============================] - 6s 5ms/step - loss: 0.1170 - categorical_accuracy: 0.9679
Epoch 3/50
1198/1198 [==============================] - 7s 6ms/step - loss: 0.0878 - categorical_accuracy: 0.9753
Epoch 4/50
1198/1198 [==============================] - 5s 5ms/step - loss: 0.0705 - categorical_accuracy: 0.9800
Epoch 5/50
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0595 - categorical_accuracy: 0.9831
Epoch 6/50
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0516 - categorical_accuracy: 0.9846
Epoch 7/50
1198/1198 [==============================] - 7s 6ms/step - loss: 0.0448 - categorical_accuracy: 0.9871
Epoch 8/50
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0394 - categorical_accuracy: 0.9882
Epoch 9/50
1198/1198 [==============================] - 5s 5ms/step - loss: 0.0353 - ca

In [ ]:
model.save("centralized_model.h5")